In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum, count, avg, round, udf, corr
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, IndexToString, StandardScaler
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.types import DoubleType
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import VectorUDT


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
spark = SparkSession.builder\
    .appName("Préparation de données")\
    .enableHiveSupport()\
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/18 13:24:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/18 13:24:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


DataFrame[]

In [3]:
clients_immatriculations = spark.sql("SELECT * FROM clients_immatriculations")
clients_immatriculations.printSchema()

root
 |-- immatriculation: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sexe: string (nullable = true)
 |-- taux: double (nullable = true)
 |-- situationfamiliale: string (nullable = true)
 |-- nbenfantacharge: integer (nullable = true)
 |-- deuxiemevoiture: boolean (nullable = true)
 |-- taux_eligible: boolean (nullable = true)
 |-- marque: string (nullable = true)
 |-- modele: string (nullable = true)
 |-- puissance: integer (nullable = true)
 |-- longueur: string (nullable = true)
 |-- nbplaces: integer (nullable = true)
 |-- nbportes: integer (nullable = true)
 |-- couleur: string (nullable = true)
 |-- occasion: boolean (nullable = true)
 |-- prix: integer (nullable = true)
 |-- categorie: string (nullable = true)



### Verification des doublons

In [4]:
doublons = clients_immatriculations.groupBy(*clients_immatriculations.columns).agg(count("*").alias("count")).filter("count > 1")
doublons.show(truncate=False)

+---------------+---+----+----+------------------+---------------+---------------+-------------+------+------+---------+--------+--------+--------+-------+--------+----+---------+-----+
|immatriculation|age|sexe|taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|taux_eligible|marque|modele|puissance|longueur|nbplaces|nbportes|couleur|occasion|prix|categorie|count|
+---------------+---+----+----+------------------+---------------+---------------+-------------+------+------+---------+--------+--------+--------+-------+--------+----+---------+-----+
+---------------+---+----+----+------------------+---------------+---------------+-------------+------+------+---------+--------+--------+--------+-------+--------+----+---------+-----+



### Analyse des **null**

In [5]:
doublons = clients_immatriculations.groupBy('nbenfantacharge').count().show()

+---------------+-----+
|nbenfantacharge|count|
+---------------+-----+
|              1|16425|
|              3|11451|
|              4| 9960|
|              2|16592|
|              0|44973|
+---------------+-----+



In [6]:
clients_immatriculations = clients_immatriculations.fillna(0)

### Supprimer les colones innutiles

In [7]:
clients_immatriculations = clients_immatriculations.drop('immatriculation')
clients_immatriculations = clients_immatriculations.drop('couleur')

### OneHotEncoder

In [8]:
indexer_marque = StringIndexer(inputCol="marque", outputCol="marque_index")
indexer_marque = indexer_marque.fit(clients_immatriculations)
clients_immatriculations = indexer_marque.transform(clients_immatriculations)

encoder_marque = OneHotEncoder(inputCol="marque_index", outputCol="marque_encoded")
clients_immatriculations = encoder_marque.fit(clients_immatriculations).transform(clients_immatriculations)

In [9]:
indexer_sexe = StringIndexer(inputCol="sexe", outputCol="sexe_index")
indexer_sexe = indexer_sexe.fit(clients_immatriculations)
clients_immatriculations = indexer_sexe.transform(clients_immatriculations)

encoder_sexe = OneHotEncoder(inputCol="sexe_index", outputCol="sexe_encoded")
clients_immatriculations = encoder_sexe.fit(clients_immatriculations).transform(clients_immatriculations)

In [10]:
indexer_situationfamiliale = StringIndexer(inputCol="situationfamiliale", outputCol="situationfamiliale_index")
indexer_situationfamiliale = indexer_situationfamiliale.fit(clients_immatriculations)
clients_immatriculations = indexer_situationfamiliale.transform(clients_immatriculations)

encoder_situationfamiliale = OneHotEncoder(inputCol="situationfamiliale_index", outputCol="situationfamiliale_encoded")
clients_immatriculations = encoder_situationfamiliale.fit(clients_immatriculations).transform(clients_immatriculations)

In [11]:
clients_immatriculations = clients_immatriculations.drop('marque') 
clients_immatriculations = clients_immatriculations.drop('marque_index')
clients_immatriculations = clients_immatriculations.drop('sexe') 
clients_immatriculations = clients_immatriculations.drop('sexe_index')
clients_immatriculations = clients_immatriculations.drop('situationfamiliale') 
clients_immatriculations = clients_immatriculations.drop('situationfamiliale_index')

clients_immatriculations = clients_immatriculations.withColumnRenamed('marque_encoded', 'marque')
clients_immatriculations = clients_immatriculations.withColumnRenamed('sexe_encoded', 'sexe')
clients_immatriculations = clients_immatriculations.withColumnRenamed('situationfamiliale_encoded', 'situationfamiliale')

## Changer des Boolean en Int

In [12]:
clients_immatriculations = clients_immatriculations.withColumn(
    "deuxiemevoiture",
    when(col("deuxiemevoiture") == False, 0)
    .when(col("deuxiemevoiture") == True, 1)
    .otherwise(col("deuxiemevoiture").cast("int"))
)

clients_immatriculations = clients_immatriculations.withColumn(
    "taux_eligible",
    when(col("taux_eligible") == False, 0)
    .when(col("taux_eligible") == True, 1)
    .otherwise(col("taux_eligible").cast("int"))
)

clients_immatriculations = clients_immatriculations.withColumn(
    "occasion",
    when(col("occasion") == False, 0)
    .when(col("occasion") == True, 1)
    .otherwise(col("occasion").cast("int"))
)

clients_immatriculations.show(4)

+----+-----+---------------+---------------+-------------+----------------+---------+-----------+--------+--------+--------+-----+-------------------+---------------+-------------+------------------+
| age| taux|nbenfantacharge|deuxiemevoiture|taux_eligible|          modele|puissance|   longueur|nbplaces|nbportes|occasion| prix|          categorie|         marque|         sexe|situationfamiliale|
+----+-----+---------------+---------------+-------------+----------------+---------+-----------+--------+--------+--------+-----+-------------------+---------------+-------------+------------------+
|61.0|188.0|              0|              0|            0|     picanto 1.1|       65|     courte|       5|       5|       0| 8990|citadine economique|(18,[13],[1.0])|(1,[0],[1.0])|     (3,[1],[1.0])|
|50.0|460.0|              3|              0|            0|vel satis 3.5 v6|      245|tres longue|       5|       5|       0|49200|      suv/crossover| (18,[2],[1.0])|    (1,[],[])|     (3,[0],[1.0])|


### Analyse de **Modele**

In [13]:
clients_immatriculations.show()

+----+------+---------------+---------------+-------------+----------------+---------+-----------+--------+--------+--------+------+-------------------+---------------+-------------+------------------+
| age|  taux|nbenfantacharge|deuxiemevoiture|taux_eligible|          modele|puissance|   longueur|nbplaces|nbportes|occasion|  prix|          categorie|         marque|         sexe|situationfamiliale|
+----+------+---------------+---------------+-------------+----------------+---------+-----------+--------+--------+--------+------+-------------------+---------------+-------------+------------------+
|61.0| 188.0|              0|              0|            0|     picanto 1.1|       65|     courte|       5|       5|       0|  8990|citadine economique|(18,[13],[1.0])|(1,[0],[1.0])|     (3,[1],[1.0])|
|50.0| 460.0|              3|              0|            0|vel satis 3.5 v6|      245|tres longue|       5|       5|       0| 49200|      suv/crossover| (18,[2],[1.0])|    (1,[],[])|     (3,[0

In [14]:
count_modele = clients_immatriculations.select("modele").distinct().count()
print(f"Il y a {count_modele} modeles.")

Il y a 28 modeles.


On travail uniquement sur les catégories, les modèles spécifiques n'ajoutent pas de valeur.
L'analyse est simplifier en se concentrant sur des regroupements plus larges.

In [15]:
indexer = StringIndexer(inputCol="categorie", outputCol="categorie_indexed")
indexer_model = indexer.fit(clients_immatriculations)
clients_immatriculations = indexer_model.transform(clients_immatriculations)

clients_immatriculations = clients_immatriculations.drop('modele') 
clients_immatriculations = clients_immatriculations.drop('categorie') 

clients_immatriculations = clients_immatriculations.withColumnRenamed('categorie_indexed', 'categorie')

clients_immatriculations.show(n=10)

+----+------+---------------+---------------+-------------+---------+-----------+--------+--------+--------+------+---------------+-------------+------------------+---------+
| age|  taux|nbenfantacharge|deuxiemevoiture|taux_eligible|puissance|   longueur|nbplaces|nbportes|occasion|  prix|         marque|         sexe|situationfamiliale|categorie|
+----+------+---------------+---------------+-------------+---------+-----------+--------+--------+--------+------+---------------+-------------+------------------+---------+
|61.0| 188.0|              0|              0|            0|       65|     courte|       5|       5|       0|  8990|(18,[13],[1.0])|(1,[0],[1.0])|     (3,[1],[1.0])|      2.0|
|50.0| 460.0|              3|              0|            0|      245|tres longue|       5|       5|       0| 49200| (18,[2],[1.0])|    (1,[],[])|     (3,[0],[1.0])|      0.0|
|54.0| 403.0|              0|              0|            0|       55|     courte|       5|       3|       0| 12200| (18,[4],[

### Analyse de **longueur**

In [16]:
clients_immatriculations.select("longueur").distinct().show()

+-----------+
|   longueur|
+-----------+
|    moyenne|
|tres longue|
|     courte|
|     longue|
+-----------+



In [17]:
indexer_longueur = StringIndexer(inputCol="longueur", outputCol="longueur_indexed")
indexer_model_longueur = indexer_longueur.fit(clients_immatriculations)
clients_immatriculations = indexer_model_longueur.transform(clients_immatriculations)

clients_immatriculations = clients_immatriculations.drop('longueur')
clients_immatriculations = clients_immatriculations.withColumnRenamed('longueur_indexed', 'longueur')

clients_immatriculations.show(n=4)

+----+-----+---------------+---------------+-------------+---------+--------+--------+--------+-----+---------------+-------------+------------------+---------+--------+
| age| taux|nbenfantacharge|deuxiemevoiture|taux_eligible|puissance|nbplaces|nbportes|occasion| prix|         marque|         sexe|situationfamiliale|categorie|longueur|
+----+-----+---------------+---------------+-------------+---------+--------+--------+--------+-----+---------------+-------------+------------------+---------+--------+
|61.0|188.0|              0|              0|            0|       65|       5|       5|       0| 8990|(18,[13],[1.0])|(1,[0],[1.0])|     (3,[1],[1.0])|      2.0|     1.0|
|50.0|460.0|              3|              0|            0|      245|       5|       5|       0|49200| (18,[2],[1.0])|    (1,[],[])|     (3,[0],[1.0])|      0.0|     0.0|
|54.0|403.0|              0|              0|            0|       55|       5|       3|       0|12200| (18,[4],[1.0])|(1,[0],[1.0])|     (3,[1],[1.0])|

### Nombre de place

In [18]:
clients_immatriculations.groupBy("nbplaces").count().show()

+--------+-----+
|nbplaces|count|
+--------+-----+
|       5|99401|
+--------+-----+



Nombre de place ne dispose d'aucune variation, elle n'apporte rien au modele.

In [19]:
clients_immatriculations = clients_immatriculations.drop('nbplaces')

## Normalisation

In [20]:
clients_immatriculations.groupBy("categorie").count().show()

+---------+-----+
|categorie|count|
+---------+-----+
|      0.0|36467|
|      1.0|30361|
|      3.0| 5795|
|      2.0|26778|
+---------+-----+



In [21]:
clients_immatriculations.printSchema()
clients_immatriculations.show()

root
 |-- age: double (nullable = false)
 |-- taux: double (nullable = false)
 |-- nbenfantacharge: integer (nullable = true)
 |-- deuxiemevoiture: integer (nullable = true)
 |-- taux_eligible: integer (nullable = true)
 |-- puissance: integer (nullable = true)
 |-- nbportes: integer (nullable = true)
 |-- occasion: integer (nullable = true)
 |-- prix: integer (nullable = true)
 |-- marque: vector (nullable = true)
 |-- sexe: vector (nullable = true)
 |-- situationfamiliale: vector (nullable = true)
 |-- categorie: double (nullable = false)
 |-- longueur: double (nullable = false)

+----+------+---------------+---------------+-------------+---------+--------+--------+------+---------------+-------------+------------------+---------+--------+
| age|  taux|nbenfantacharge|deuxiemevoiture|taux_eligible|puissance|nbportes|occasion|  prix|         marque|         sexe|situationfamiliale|categorie|longueur|
+----+------+---------------+---------------+-------------+---------+--------+-------

## Classificateur

In [22]:
# Étape 1 : Identifier les colonnes numériques
numerical_cols = ['age', 'taux', 'nbenfantacharge', 'puissance', 'nbportes', 'prix', 'categorie', 'longueur']

# Étape 2 : Assembler directement les colonnes numériques et catégorielles
categorical_vector_cols = ['marque', 'sexe', 'situationfamiliale']

final_assembler = VectorAssembler(
    inputCols=numerical_cols + categorical_vector_cols,
    outputCol="features"
)
df_final = final_assembler.transform(clients_immatriculations)

# Afficher les résultats
df_final.select("features").show(truncate=False)


+--------------------------------------------------------------------------------------+
|features                                                                              |
+--------------------------------------------------------------------------------------+
|(30,[0,1,3,4,5,6,7,21,26,28],[61.0,188.0,65.0,5.0,8990.0,2.0,1.0,1.0,1.0,1.0])        |
|(30,[0,1,2,3,4,5,10,27],[50.0,460.0,3.0,245.0,5.0,49200.0,1.0,1.0])                   |
|(30,[0,1,3,4,5,6,7,12,26,28],[54.0,403.0,55.0,3.0,12200.0,2.0,1.0,1.0,1.0,1.0])       |
|(30,[0,1,2,3,4,5,13,26,27],[74.0,531.0,4.0,306.0,5.0,70910.0,1.0,1.0,1.0])            |
|(30,[0,1,3,4,5,6,7,12,26,28],[35.0,208.0,150.0,5.0,16029.0,1.0,3.0,1.0,1.0,1.0])      |
|(30,[0,1,2,3,4,5,7,16,26,27],[66.0,960.0,1.0,150.0,5.0,38600.0,2.0,1.0,1.0,1.0])      |
|(30,[0,1,3,4,5,6,7,23,27],[65.0,199.0,102.0,5.0,18880.0,1.0,2.0,1.0,1.0])             |
|(30,[0,1,3,4,5,13,26,27],[67.0,1072.0,306.0,5.0,101300.0,1.0,1.0,1.0])                |
|(30,[0,1,2,3,4,5,10,

Entrainement du modele

In [23]:
(trainingData, testData) = df_final.randomSplit([0.8, 0.2], seed=42)

rf = RandomForestClassifier(labelCol="categorie", featuresCol="features", numTrees=20)
model = rf.fit(trainingData)

Évaluation initiale

In [24]:
predictions = model.transform(testData)

predictions.show()

evaluator = MulticlassClassificationEvaluator(
    labelCol="categorie", predictionCol="prediction", metricName="accuracy"
)

accuracy = evaluator.evaluate(predictions)
print(f"Précision du modèle sur l'ensemble de test = {accuracy * 100:.2f}%")

+----+-----+---------------+---------------+-------------+---------+--------+--------+-----+---------------+-------------+------------------+---------+--------+--------------------+--------------------+--------------------+----------+
| age| taux|nbenfantacharge|deuxiemevoiture|taux_eligible|puissance|nbportes|occasion| prix|         marque|         sexe|situationfamiliale|categorie|longueur|            features|       rawPrediction|         probability|prediction|
+----+-----+---------------+---------------+-------------+---------+--------+--------+-----+---------------+-------------+------------------+---------+--------+--------------------+--------------------+--------------------+----------+
|18.0|156.0|              3|              0|            0|      200|       5|       0|30000|(18,[11],[1.0])|    (1,[],[])|     (3,[0],[1.0])|      3.0|     0.0|(30,[0,1,2,3,4,5,...|[0.43259532539156...|[0.02162976626957...|       3.0|
|18.0|161.0|              0|              0|            0|  

F1-Score

In [25]:
f1_evaluator = MulticlassClassificationEvaluator(
    labelCol="categorie", predictionCol="prediction", metricName="f1"
)
f1_score = f1_evaluator.evaluate(predictions)
print(f"F1-score global : {f1_score * 100:.2f}%")

F1-score global : 100.00%


Matrice de confusion

In [26]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedCategorie", labels=indexer_model.labels)
predictions = labelConverter.transform(predictions)

# Regroupement
predictions.groupBy("categorie", "predictedCategorie").count().show()

+---------+-------------------+-----+
|categorie| predictedCategorie|count|
+---------+-------------------+-----+
|      2.0|citadine economique| 5272|
|      3.0|              autre| 1117|
|      0.0|      suv/crossover| 7230|
|      1.0|          familiale| 5983|
+---------+-------------------+-----+



Validation croisée

Vérification des corrélations entre caractéristiques et catégorie

In [27]:
df_final.select("scaled_numerical_features").show(5, truncate=False)

AnalysisException: cannot resolve 'scaled_numerical_features' given input columns: [age, categorie, deuxiemevoiture, features, longueur, marque, nbenfantacharge, nbportes, occasion, prix, puissance, sexe, situationfamiliale, taux, taux_eligible];
'Project ['scaled_numerical_features]
+- Project [age#205, taux#206, nbenfantacharge#207, deuxiemevoiture#857, taux_eligible#874, puissance#208, nbportes#210, occasion#891, prix#211, marque#806, sexe#823, situationfamiliale#840, categorie#1187, longueur#1395, UDF(struct(age, age#205, taux, taux#206, nbenfantacharge_double_VectorAssembler_4894c645e38d, cast(nbenfantacharge#207 as double), puissance_double_VectorAssembler_4894c645e38d, cast(puissance#208 as double), nbportes_double_VectorAssembler_4894c645e38d, cast(nbportes#210 as double), prix_double_VectorAssembler_4894c645e38d, cast(prix#211 as double), categorie, categorie#1187, longueur, longueur#1395, marque, marque#806, sexe, sexe#823, situationfamiliale, situationfamiliale#840)) AS feat

In [ ]:
clients_immatriculations.groupBy('categorie').count().show()

# Taille du jeu de données total
print(f"Taille du jeu de données : {df_final.count()}")

# Taille des ensembles d'entraînement et de test
print(f"Taille de l'ensemble d'entraînement : {trainingData.count()}")
print(f"Taille de l'ensemble de test : {testData.count()}")